<a href="https://colab.research.google.com/github/Tarwish2005/Pdf_Chat_Bot/blob/main/PDF_Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U langgraph


In [5]:
# Install required packages
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph langchain-openai pypdf

import getpass
import os
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

# Set up environment
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"


if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

# Initialize LLM and embeddings
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings

llm = init_chat_model("gpt-4o-mini", model_provider="openai")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Define the Search schema
class Search(BaseModel):
    """Search query for retrieving relevant documents."""
    query: str = Field(description="The search query to find relevant information")
    section: str = Field(default="", description="Optional section filter")

# Load and process PDF
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore

file_path = "/content/drive/MyDrive/BAJHLIP23020V012223.pdf"

# Load PDF pages
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)
all_splits = text_splitter.split_documents(pages)
print(f"Split PDF into {len(all_splits)} chunks.")

# Create vector store
vector_store = InMemoryVectorStore.from_documents(all_splits, embeddings)

# Set up RAG prompt
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END, START


prompt = hub.pull("rlm/rag-prompt")

# Define state
class State(TypedDict):
    question: str
    query: Search
    context: List[Document]
    answer: str

# Define workflow functions
def analyze_query(state: State):
    structured_llm = llm.with_structured_output(Search)
    query = structured_llm.invoke(state["question"])
    return {"query": query}

def retrieve(state: State):
    query = state["query"]
    # Simple similarity search without section filtering
    # (since PDF loader doesn't typically add section metadata)
    retrieved_docs = vector_store.similarity_search(query.query, k=4)
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Build the graph
graph_builder = StateGraph(State).add_sequence([analyze_query, retrieve, generate])
graph_builder.add_edge(START, "analyze_query")
graph = graph_builder.compile()

# Test the system
def ask_question(question: str):
    result = graph.invoke({"question": question})
    return result["answer"]



Split PDF into 257 chunks.


In [7]:
question = ""
answer = ask_question(question)
print(answer)

I don't know.
